In [1]:
import datetime
import time
import sys
sys.path.append(r'/home/hengtian/lucy/factor-analyzer-v4/')
import pymysql
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import multiprocessing
import os
import queue
import tqdm
# from src import data 
# from src import analyzer


import datetime as dt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import cufflinks as cf
cf.go_offline()

sys.path.append(os.path.abspath('/home/hengtian/lucy/PortfolioOptimizer/'))
from PortfolioOptimizerMaster.src import tools

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression

# 月初对齐

In [182]:
high = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/high_limit.csv", index_col = 0)
high.index = pd.to_datetime(high.index).date
high = high[op_post.columns]

In [183]:
op = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/open_price.csv", index_col = 0)
op.index = pd.to_datetime(op.index).date
op = op[op_post.columns]

In [155]:
op_post = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/open_price_post.csv",index_col = 0)
op_post.index = pd.to_datetime(op_post.index).date

In [156]:
trading_days = op_post.index

In [285]:
d1 = { }
for date in trading_days:
    if date.strftime("%Y%m") not in d1.keys():
        d1[date.strftime("%Y%m")] = []
    d1[date.strftime("%Y%m")].append(date)

In [166]:
op_post= op_post.mask(op_post==0,np.nan)

In [289]:
ym_l = sorted(list(d1.keys()))

In [302]:
d = list(d1.get(ym_l[0]))[1]
dp = list(d1.get(ym_l[1]))[1]
print(dp)
pre_price = op_post.loc[d]
# print(pre_price)
paused = high.loc[d]==op.loc[d]
# print(paused)
pre_price = pre_price.mask(paused,np.nan)
s = paused[paused ==True].index
print(s)
post = op_post.loc[dp]
avg = post/pre_price-1
print(np.nanmean(avg))
print(avg.dropna())
# l = [high.loc[d]==op.loc[d]]

2013-02-04
Index(['000078.XSHE', '000409.XSHE', '000504.XSHE', '000677.XSHE',
       '002166.XSHE', '300239.XSHE', '300294.XSHE', '600784.XSHG'],
      dtype='object')
0.07829565469879404
000001.XSHE    0.392990
000004.XSHE    0.139155
000005.XSHE    0.040536
000006.XSHE    0.130616
000007.XSHE    0.393666
                 ...   
603333.XSHG   -0.005545
603366.XSHG   -0.016015
603399.XSHG    0.165574
603766.XSHG    0.052273
603993.XSHG    0.239752
Length: 2365, dtype: float64


In [314]:
i = 1
d = { }
while i<len(ym_l):
    #前一个月取第一个交易日
    s =[]
    for j in range(len(list(d1.get(ym_l[i-1])))):
        pre_date =list(d1.get(ym_l[i-1]))[j]
    #取后一个月第一个交易日
        try:
            post_date = list(d1.get(ym_l[i]))[j]
        except IndexError:
            post_date = list(d1.get(ym_l[i]))[-1]
    #前一个月交易日所有价格
        pre_price = op_post.loc[pre_date]
    #前一个月所有开盘涨停的股票==Nan
        paused = high.loc[pre_date]==op.loc[pre_date]
        pre_price = pre_price.mask(paused,np.nan)
    #后一个月交易日所有价格
        post_price = op_post.loc[post_date]
        avg = np.nanmean(post_price/pre_price-1)
        s.append(avg)
    d[ym_l[i]] =s
    i+=1

In [320]:
# df= pd.DataFrame(np.empty((len(ym_l)-1,25),dtype=object),columns =np.arange(1,26),index = list(d.keys()))
s = []
for i in range(len(d)):
    month = list(d.keys())[i]
    length = len(d.get(month))
    s.append([monthlength])
#     exd =[np.nan]*(23-len(d.get(month)))
#     d.get(month).extend(exd)

In [311]:
temp = pd.DataFrame(d)
temp.index= np.arange(1,24)

In [312]:
temp.mean(axis=1).iplot()

In [337]:
temp

,201302,201303,201304,201305,201306,201307,201308,201309,201310,201311,...,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107
1,0.052451,0.039819,-0.036991,-0.032677,0.148853,-0.160195,0.074980,0.082869,0.055243,-0.030677,...,-0.054965,-0.011238,0.027276,-0.033190,-0.070838,0.039486,0.013483,0.007131,0.066361,0.034613
2,0.078296,0.032193,-0.034210,-0.029422,0.129395,-0.133981,0.082184,0.057708,0.061864,-0.043985,...,-0.043248,-0.027021,0.039363,-0.037753,-0.072046,0.050816,-0.003229,0.006812,0.068092,0.007433
3,0.047506,0.021110,-0.018759,0.001333,0.084024,-0.103817,0.069244,0.072936,0.061420,-0.054251,...,-0.024791,-0.032558,0.021405,-0.036813,-0.071933,0.039792,0.011246,-0.002579,0.066386,0.012861
4,0.052685,0.032831,-0.066653,0.041568,0.071018,-0.110132,0.090325,0.056409,0.057401,-0.037368,...,0.003617,-0.032118,0.013946,-0.048291,-0.074071,0.069485,0.009879,-0.010881,0.063074,0.030204
5,0.055414,0.042124,-0.066329,0.030850,0.052104,-0.084737,0.083489,0.046521,0.070511,-0.058817,...,-0.020111,-0.016301,0.006497,-0.075464,-0.062265,0.075271,0.025960,-0.012097,0.063425,0.018057
6,0.050610,0.027475,-0.049901,0.029005,0.014852,-0.079153,0.093320,0.070617,0.069729,-0.080471,...,-0.017537,-0.014795,0.006834,-0.071100,-0.074916,0.109596,-0.000675,0.002849,0.063227,0.010342
7,0.094937,0.001051,-0.041846,0.029367,-0.001915,-0.090300,0.136770,0.070458,0.056975,-0.087659,...,-0.008920,-0.001138,-0.013524,-0.092897,-0.049886,0.087756,0.020997,0.005928,0.055590,0.012519
8,0.055283,-0.000189,-0.050761,0.047839,0.007206,-0.100632,0.131231,0.076628,0.025728,-0.058832,...,-0.008844,-0.004337,-0.021905,-0.061328,-0.043710,0.050158,0.024844,0.028072,0.050046,0.008997
9,0.021569,-0.013372,-0.033221,0.062885,0.005478,-0.074062,0.110985,0.048632,0.035454,-0.048803,...,0.060249,-0.024174,-0.010657,-0.081417,-0.005354,0.011095,0.034701,0.027129,0.066565,0.002639
10,0.039252,-0.034954,-0.044084,0.073198,0.013693,-0.048431,0.090686,0.045400,0.039261,-0.072683,...,0.012799,-0.010589,-0.026436,-0.060576,-0.001876,0.019430,0.025140,0.023358,0.052510,0.009996


In [323]:
s= np.array(s)
len(s[s<20])

27

# 月末对齐

In [338]:
i = 1
d_last = { }
while i<len(ym_l):
    s =[]
    j = len(list(d1.get(ym_l[i-1])))-1
    while j>=0:
         #前一个月取第最后一个交易日
        pre_date =list(d1.get(ym_l[i-1]))[j]
    #取后一个月第一个交易日
        try:
            post_date = list(d1.get(ym_l[i]))[j]
        except IndexError:
            post_date = list(d1.get(ym_l[i]))[-1]
    #前一个月交易日所有价格
        pre_price = op_post.loc[pre_date]
    #前一个月所有开盘涨停的股票==Nan
        paused = high.loc[pre_date]==op.loc[pre_date]
        pre_price = pre_price.mask(paused,np.nan)
    #后一个月交易日所有价格
        post_price = op_post.loc[post_date]
        avg = np.nanmean(post_price/pre_price-1)
        s.append(avg)
        j-=1
    d_last[ym_l[i]] =s
    i+=1

In [336]:
pre = list(d1.get(ym_l[0]))[-1]
post= list(d1.get(ym_l[1]))[-1]
print(dp)
pre_price = op_post.loc[pre]
# print(pre_price)
paused = high.loc[pre]==op.loc[pre]
# print(paused)
pre_price = pre_price.mask(paused,np.nan)
s = paused[paused ==True].index
print(s)
pp = op_post.loc[post]
avg = pp/pre_price-1
print(np.nanmean(avg))
print(avg.dropna())

2013-02-04
Index(['000513.XSHE', '000698.XSHE', '000720.XSHE', '000901.XSHE',
       '300006.XSHE', '600136.XSHG', '600681.XSHG', '600806.XSHG',
       '600814.XSHG'],
      dtype='object')
0.015796495991281443
000001.XSHE   -0.006112
000002.XSHE   -0.085714
000004.XSHE    0.077007
000005.XSHE   -0.078110
000006.XSHE   -0.102474
                 ...   
603333.XSHG   -0.001862
603366.XSHG    0.078413
603399.XSHG   -0.073466
603766.XSHG   -0.076759
603993.XSHG   -0.043749
Length: 2365, dtype: float64


In [340]:
for i in range(len(d_last)):
    month = list(d_last.keys())[i]
#     length = len(d_last.get(month))
#     s.append([month,length])
    exd =[np.nan]*(23-len(d_last.get(month)))
    d_last.get(month).extend(exd)

In [342]:
temp_last = pd.DataFrame(d_last)

In [345]:
temp_last.index= np.arange(1,24)
temp_last.mean(axis=1).iplot()

In [362]:
temp_last.to_csv("month_月末对齐.csv")

# 指数趋势

## 月初对齐

In [347]:
bmk = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/bmk_open.csv", index_col = 0)
bmk.index = pd.to_datetime(bmk.index).date

In [365]:
bmk

,000001.XSHG,000016.XSHG,000300.XSHG,000905.XSHG,399001.XSHE,399005.XSHE,399006.XSHE
2013-01-04,2289.5100,1885.9640,2551.8100,3297.0430,9204.114,4253.417,717.303
2013-01-07,2271.6630,1865.0580,2518.0500,3253.6740,9073.968,4163.785,704.372
2013-01-08,2284.6540,1879.8480,2534.6500,3288.1250,9102.033,4205.756,718.431
2013-01-09,2271.2990,1851.6070,2518.1000,3321.7650,9077.499,4273.294,732.011
2013-01-10,2274.3770,1854.9870,2525.5900,3335.0880,9151.813,4308.620,734.309
...,...,...,...,...,...,...,...
2021-07-02,3569.8919,3495.5976,5186.6613,6701.0642,14937.733,9704.845,3429.606
2021-07-05,3516.9228,3402.1090,5073.7787,6670.1262,14666.681,9583.297,3332.490
2021-07-06,3533.1304,3405.4701,5089.7780,6727.9674,14745.453,9697.706,3359.262
2021-07-07,3508.5901,3384.5665,5052.4204,6661.8535,14592.750,9579.829,3278.125


In [408]:
zz = bmk['000001.XSHG']

In [409]:
trad_d = zz.index
d_zz = { }
for date in trad_d:
    if date.strftime("%Y%m") not in d_zz.keys():
        d_zz[date.strftime("%Y%m")] = []
    d_zz[date.strftime("%Y%m")].append(date)

In [410]:
ym = sorted(list(d_zz.keys()))

In [411]:
i = 1
d_bmk = { }
while i<len(ym):
    #前一个月取第一个交易日
    s =[]
    for j in range(len(list(d_zz.get(ym[i-1])))):
        pre_date =list(d_zz.get(ym[i-1]))[j]
    #取后一个月第一个交易日
        try:
            post_date = list(d_zz.get(ym[i]))[j]
        except IndexError:
            post_date = list(d_zz.get(ym[i]))[-1]
    #前一个月交易日指数
        pre_price = zz.loc[pre_date]
    #后一个月交易日指数
        post_price = zz.loc[post_date]
#         avg = np.nanmean(post_price/pre_price-1)
        s.append(post_price/pre_price-1)
    d_bmk[ym[i]] =s
    i+=1

In [412]:
for i in range(len(d_bmk)):
    month = list(d_bmk.keys())[i]
#     length = len(d_bmk.get(month))
#     s.append([month,length])
    exd =[np.nan]*(23-len(d_bmk.get(month)))
    d_bmk.get(month).extend(exd)

In [413]:
temp_bmk = pd.DataFrame(d_bmk)
temp_bmk.index= np.arange(1,24)
temp_bmk.mean(axis=1).iplot()

In [414]:
temp_bmk.to_csv("bmk_month_000001.csv")

## 月末对齐

In [415]:
i = 1
d_bmk_l = { }
while i<len(ym):
    s =[]
    j = len(list(d_zz.get(ym[i-1])))-1
    while j>=0:
         #前一个月取第最后一个交易日
        pre_date =list(d_zz.get(ym[i-1]))[j]
    #取后一个月第一个交易日
        try:
            post_date = list(d_zz.get(ym[i]))[j]
        except IndexError:
            post_date = list(d_zz.get(ym[i]))[-1]
    #前一个月交易日所有价格
        pre_price = zz.loc[pre_date]
    #后一个月交易日所有价格
        post_price = zz.loc[post_date]
        s.append(post_price/pre_price-1)
        j-=1
    d_bmk_l[ym[i]] =s
    i+=1

In [416]:
for i in range(len(d_bmk_l)):
    month = list(d_bmk_l.keys())[i]
#     length = len(d_bmk_l.get(month))
#     s.append([month,length])
    exd =[np.nan]*(23-len(d_bmk_l.get(month)))
    d_bmk_l.get(month).extend(exd)

In [417]:
temp_bmk_l = pd.DataFrame(d_bmk_l)
temp_bmk_l.index= np.arange(1,24)
temp_bmk_l.mean(axis=1).iplot()

In [418]:
temp_bmk_l.to_csv("bmk_month_月末对齐(01).csv")